In [1]:
import yaml
import sys
import os
import numpy as np
import torch 
os.chdir('/home/jonfrey/PLR3')
sys.path.insert(0, os.getcwd())
sys.path.append(os.path.join(os.getcwd() + '/src'))
sys.path.append(os.path.join(os.getcwd() + '/lib'))

import loaders_v2
from loaders_v2 import GenericDataset
from rotations import * 
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from visu import plot_pcd, Visualizer
import copy
from scipy.spatial.transform import Rotation as R
from helper import re_quat
from PIL import Image, ImageDraw
from deep_im import LossAddS
import copy
#from deep_im import flow_to_trafo
from visu import Visualizer
import matplotlib.pyplot as plt
import k3d
exp_cfg_path = '/home/jonfrey/PLR3/yaml/exp/exp_ws_deepim_debug_natrix.yml'
exp_cfg_path = '/home/jonfrey/PLR3/yaml/exp/exp_evaluate_pose_estimation.yml'
env_cfg_path = '/home/jonfrey/PLR3/yaml/env/env_natrix_jonas.yml'

h = 480
w = 640


def load_from_file(p):
    if os.path.isfile(p):
        with open(p, 'r') as f:
            data = yaml.safe_load(f)
    else:
        raise ValueError
    return data

exp = load_from_file(exp_cfg_path)
env = load_from_file(env_cfg_path)

dataset_train = GenericDataset(
    cfg_d=exp['d_train'],
    cfg_env=env)

batch = dataset_train[13450][0] #bann 10450
points, choose, img, target, model_points, idx = batch[0:6]
depth_img, label_img, img_orig, cam = batch[6:10]
gt_rot_wxyz, gt_trans, unique_desig = batch[10:13]

real_img, render_img, real_d, render_d, gt_label_cropped = batch[13:18]
pred_rot_wxyz, pred_trans, pred_points, h_render,h_real, render_img_original = batch[18:24]
u_map, v_map, flow_mask,  bb = batch[24:]



                

/home/jonfrey/PLR3/src/loaders_v2/dataset_ycb.py:426: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  init_trans = torch.normal(mean=torch.tensor(gt_trans), std=nt)
/home/jonfrey/PLR3/src/helper/bounding_box.py:203: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629427478/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  masked_idx = (d != 0).nonzero()
/home/jonfrey/miniconda3/envs/track_latest/lib/python3.7/site-packages/torch/nn/functional.py:3121: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the 

In [2]:
class Drawer():
    def __init__(self):
        self.im_in_plot = 0
        self.data = []
        
    def disp_img_1d(self,img,hold=False, save=False, nr=0 , ret=False):
        self.data.append(img)
        p = '/home/jonfrey/Debug/Midterm2/'
        
        if not hold:
            fig = plt.figure(figsize=(6*2*len(self.data),7))
            ax = []
            for j,a in enumerate(self.data):
                ax.append( fig.add_subplot(1,len(self.data), j+1)  )
                
                ax[-1].get_xaxis().set_visible(False)
                ax[-1].get_yaxis().set_visible(False)
                pos = ax[-1].imshow( a, cmap='Reds' )
                
                fig.colorbar(pos, ax=ax[-1])
            plt.show()
            if save:
                fig.savefig(p+str(nr)+'.png', dpi=300)
                
            if ret: 
                if isinstance( self.data[0], torch.Tensor):
                    self.data[0] = self.data[0].numpy()
                    print('CONV')
                    
                print(self.data[0].shape)
                a = np.max(self.data[0])
                b = np.min(self.data[0])
                
                d = (self.data[0]-float(b))
                d = (d / ((float(a)-float(b))) )*255 
                d = np.uint8(d)
                img = Image.fromarray( d )
                return d
            self.data = []
            self.ax = []
            
     

In [3]:
desig = unique_desig[0]

# [('data/0003/001742',), tensor([8], dtype=torch.int32)]

desig = 'data/0003/001742'

_p_ycb = "/media/scratch1/jonfrey/datasets/YCB_Video_Dataset"
depth = np.array(Image.open(
    '{0}/{1}-depth.png'.format(_p_ycb, desig)))
depth.shape

label = np.array(Image.open(
    '{0}/{1}-label.png'.format(_p_ycb, desig)))
img = np.array(Image.open(
    '{0}/{1}-color.png'.format(_p_ycb, desig)))

Nc = 256
cmap = plt.cm.get_cmap('gist_rainbow', Nc)
cmaplist = [cmap(i) for i in range(cmap.N)]

def disp_alignment(depth, label, real):
    data = np.zeros((480,640,4), dtype=np.uint8)
    data_depth = np.zeros((480,640,4), dtype=np.uint8)
    t = real
    data[:,:,:3] = t.numpy() # red patch in upper left
    data_depth[:,:,:3] = t.numpy()
    data[:,:,3] = 70
    data[:,:,3][label==8] = 255
    
    min_val = torch.min( depth[depth!=0] )
    max_val = torch.max( depth[depth!=0] )
    val = torch.clamp( ((depth-min_val) // (max_val-min_val))*255, 0, 255)
    
    img = Image.fromarray(data, 'RGBA')
    display(img)

def plot_mask(mask):
    min_val = torch.min( mask )
    max_val = float( max(1,torch.max( mask )) )
    mask = torch.clamp( (mask-min_val) / (max_val-min_val)*255 ,0,255)
    
    data_depth = np.zeros((480,640,4), dtype=np.uint8)
    data_depth[:,:,3] = 255
    for i in range(480):
        for j in range(640):
            data_depth[i,j,:4] = np.array( cmaplist[ int(mask[i,j])] )*255
    data_depth[:,:,3] = 255
    data_depth[:,:,3][label==2] = 255
    img_depth = Image.fromarray(data_depth, 'RGBA')
    display(img_depth)

batch = dataset_train._backend.getElement( desig, 8)
batch = batch #bann 10450   
model_points = batch[4]
idx = batch[5]  # Be carefull here the first objects starts with 0. Normally 0 is the NO object class in all other datastructures
real_img_original = batch[8]
cam = batch[9]
real_img, render_img, real_d, render_d, gt_label_cropped = batch[13:18]
pred_rot_wxyz, pred_trans, pred_points, h_render, h_real, render_img_original = batch[18:24]
u_map, v_map, flow_mask, bb = batch[24:]


In [4]:
Nc = 256
cmap = plt.cm.get_cmap('gist_rainbow', Nc)
cmaplist = [cmap(i) for i in range(cmap.N)]
#disp_alignment(real_d[0], gt_label_cropped, real_img)

In [5]:
exp['d_train']["output_cfg"]['overfitting_nr_idx'] = -1
print(exp['d_train'])
dataset_train = GenericDataset(
    cfg_d=exp['d_train'],
    cfg_env=env)
device = 'cuda'
exp['loader']['batch_size'] = 1
exp['loader']['pin_memory'] = False
exp['loader']['shuffle'] = False

dataloader_train = torch.utils.data.DataLoader(dataset_train,
                                                       **exp['loader'])
criterion_adds = LossAddS(sym_list=exp['d_train']['obj_list_sym'])

def eroision(t,size=3):
    "t: tensor shape BS, C, H,W"
    out_c = t.shape[1]
    kernel_tensor = torch.ones( (out_c,1,size,size) )
    print(size, kernel_tensor, t.shape)
    return torch.nn.functional.conv2d(t, kernel_tensor, padding=(int((size)/2), int((size)/2))) == (size*size)

def eroision_batch(t,t_size):
    "t: tensor shape BS, C, H,W"
    "t_size: tensor shape BS"
    out_c = t.shape[1]
    for b in range( t.shape[0] ):
        size = int( t_size[b] )
        kernel_tensor = torch.ones( (out_c,1,size,size) )
        t[b] = (torch.nn.functional.conv2d(t[b][None], kernel_tensor, padding=(int((size)/2), int((size)/2))) == (size*size))[0,:,:t.shape[2], :t.shape[3]]
    return t

def plot_two_pcd_line(x, y, point_size=0.005, c1='g', c2='r'):
    if c1 == 'b':
        k = 245
    elif c1 == 'g':
        k = 25811000
    elif c1 == 'r':
        k = 11801000
    elif c1 == 'black':
        k = 2580
    else:
        k = 2580

    if c2 == 'b':
        k2 = 245
    elif c2 == 'g':
        k2 = 25811000
    elif c2 == 'r':
        k2 = 11801000
    elif c2 == 'black':
        k2 = 2580
    else:
        k2 = 2580

    col1 = np.ones(x.shape[0]) * k
    col2 = np.ones(y.shape[0]) * k2
    plot = k3d.plot(name='points')
    plt_points = k3d.points(x, col1.astype(np.uint32), point_size=point_size)
    plot += plt_points
    plt_points = k3d.points(y, col2.astype(np.uint32), point_size=point_size)
    plot += plt_points
    for i in range(min(100,x.shape[0]) ):
        plot += k3d.line([x[i],y[i]],shader='mesh', width=0.0005, color=0xff0000)
    
    plt_points.shader = '3d'
    plot.display()
    




{'name': 'ycb', 'objects': 21, 'num_points': 1000, 'num_pt_mesh_small': 2300, 'num_pt_mesh_large': 2300, 'obj_list_fil': None, 'obj_list_sym': [12, 15, 18, 19, 20], 'batch_list_cfg': {'sequence_names': None, 'seq_length': 1, 'no_list_for_sequence_len_one': True, 'fixed_length': True, 'sub_sample': 1, 'mode': 'train', 'add_syn_to_train': True}, 'noise_cfg': {'status': False, 'noise_trans': 0}, 'output_cfg': {'overfitting_nr_idx': -1, 'vm_in_dataloader': True, 'noise_translation': 0.02, 'noise_rotation': 20, 'return_same_size_tensors': True, 'force_one_object_visible': True, 'status': False, 'refine': False, 'add_depth_image': True, 'add_mask_image': True, 'norm_render': False, 'color_jitter_render': {'active': False, 'cfg': [0.2, 0.2, 0.2, 0.05]}, 'norm_real': False, 'color_jitter_real': {'active': False, 'cfg': [0.2, 0.2, 0.2, 0.05]}, 'visu': {'status': True, 'return_img': True}}}


In [34]:
import torch
import numpy as np
import copy
from helper import anal_tensor


def solve_transform(keypoints, gt_keypoints):
    """
    keypoints: N x K x 3
    gt_keypoints: K x 3
    return: N x 4 x 4 transformation matrix
    """
    try:
        keypoints = keypoints.clone()
        gt_keypoints = gt_keypoints.clone()
        N, K, _ = keypoints.shape
        center = keypoints.mean(dim=1)
        gt_center = gt_keypoints.mean(dim=0)
        keypoints -= center[:, None, :]
        gt_keypoints -= gt_center[None]
        matrix = keypoints.transpose(2, 1) @ gt_keypoints[None]
        U, S, V = torch.svd(matrix)
        Vt = V.transpose(2, 1)
        Ut = U.transpose(2, 1)

        d = (V @ Ut).det()
        I = torch.eye(3, 3, dtype=gt_center.dtype, device= keypoints.device)[None].repeat(N, 1, 1)
        I[:, 2, 2] = d.clone()

        R = U @ I @ Vt
        T = torch.zeros(N, 4, 4, dtype=gt_center.dtype, device= keypoints.device)
        T[:, 0:3, 0:3] = R
        T[:, 0:3, 3] = center[None] - (R @ gt_center[None :, None])[:, :, 0]
        T[:, 3, 3] = 1.0

        return T
    except RuntimeError as error:
        import ipdb; ipdb.set_trace()
        print("Something went wrong")

        # costume implementation 
def solve_transform2(A,B):
    if A.shape[0] > B.shape[0]:
        x=torch.arange(A.shape[0],device=A.device)
        out = torch.randperm(x.numel(),device=A.device)[:B.shape[0]]
        A = torch.index_select(A, 0, out)
    if A.shape[0] < B.shape[0]:
        x=torch.arange(B.shape[0],device=A.device)
        out = torch.randperm(x.numel(),device=A.device)[:A.shape[0]]
        B = torch.index_select(B, 0, out)

    #A = torch.choice(A,B.shape[0])

    assert A.shape == B.shape

    m = A.shape[1]
    centroid_A = torch.mean(A, dim=0)
    centroid_B = torch.mean(B, dim=0)

    AA = (A - centroid_A)
    BB = (B - centroid_B)
    H = AA.transpose(0,1) @ BB
    U, S, Vt = torch.svd(H)
    R = Vt @ U.transpose(0,1)
    if torch.det(R) < 0:
        Vt[m-1,:] *= -1
        R = Vt.transpose(0,1) @ U.transpose(0,1)

    # translation
    t = centroid_B - (R @ centroid_A)
    # homogeneous transformation
    T = torch.eye(m+1, device= A.device)
    T[:m, :m] = R
    T[:m, m] = t
    return T

NR = 1000
DIM = 3
A = torch.ones( (NR,DIM), dtype= torch.float32)
B = A*1.7223
T = solve_transform2(A,B)
print(T)
def get_H(pcd):
    pcd_ret = torch.ones( (pcd.shape[0],pcd.shape[1]+1),device=pcd.device, dtype=pcd.dtype )
    pcd_ret[:,:3] = pcd
    return pcd_ret

A_hom = get_H(A)
A_hom2 = A_hom @ T.T
print(A_hom2[:,:3])

def filter_pcd_given_depthmap(pcd, depth, scal= 10000):
    """
    pcd = Nx3 troch.float32
    depth = N torch.float32

    return N torch.bool
    """
    m1 = (depth/scal) > 0.2
    #print( "Thorwn away values", (depth/scal) < 0.2 )
    return m1

    val_d = depth[ m1 ]
    mean = torch.mean(val_d)
    new_d = depth - mean
    tol = 0.5
    m2 = torch.abs( new_d/scal ) < tol 
    return m1 * m2
    
def filter_pcd( pcd, tol = 0.6):
    """
    input:
        pcd : Nx3 torch.float32
    returns:
        mask : NX3 torch.bool 
    """
    m = torch.mean(pcd, dim = 0)
    comp = m[None,:].repeat(pcd.shape[0],1) + tol
    mean_free = pcd-m[None,:].repeat(comp.shape[0],1)
    mask = torch.norm( mean_free,  dim= 1) > tol
    #print(f"filter_pcd PRE: {pcd.shape}, POST: {float(torch.sum(mask[:,None].repeat(1,3) == False ))/3.0}")
    return mask[:,None].repeat(1,3) == False

def filter_pcd_cor(pcd1, pcd2, max_mean_deviation=0.2):
    
    dif = torch.norm( pcd1-pcd2 , dim= 1)
    mean = torch.mean(dif, dim = 0)
    mean_free = torch.abs(dif-mean)
    #print(f"filter_pcd_cor PRE: {pcd1.shape[0]}, POST: {torch.sum(mean_free < max_mean_deviation)}")
    return mean_free < max_mean_deviation
    
def flow_to_trafo(*args, **kwargs):
    """
    input:
      real_br: torch.tensor torch.Size([2])
      real_tl: torch.tensor torch.Size([2])
      ren_br: torch.tensor torch.Size([2])
      ren_tl: torch.tensor torch.Size([2])
      flow_mask: torch.Size([480, 640])
      u_map: torch.Size([480, 640])
      v_map: torch.Size([480, 640])
      K_real: torch.Size([3, 3])
      K_ren: torch.Size([3, 3])
      real_d: torch.Size([480, 640]) 
      render_d: torch.Size([480, 640])
      h_real: torch.Size([4, 4])
      h_render: torch.Size([4, 4])
    output:
      P_real_in_center: torch.Size([N, 3])
      P_ren_in_center: torch.Size([N, 3]) 
      P_real_trafo: torch.Size([N, 3])
      T_res: torch.Size([4, 4])
      
      The output rotation T_res is defined in the Camera coordinate frame. 
      Therfore premultiply the T_Res with h_render to get the new h_real_new !!!
    """
    for k in kwargs.keys():
        pass
        #print(f"Variable: {k}, Type {type(kwargs[k])}, Dtype{kwargs[k].dtype}, Shape{kwargs[k].shape}")
    real_br = kwargs['real_br']
    real_tl = kwargs['real_tl']
    ren_br = kwargs['ren_br']
    ren_tl = kwargs['ren_tl']
    flow_mask = kwargs['flow_mask']
    u_map = kwargs['u_map']
    v_map = kwargs['v_map']
    K_real = kwargs['K_real']
    K_ren = kwargs['K_ren']
    real_d = kwargs['real_d']
    render_d = kwargs['render_d']
    h_real = kwargs['h_real']
    h_render = kwargs['h_render']
    # print("Flow mask input reduces to ", flow_mask.shape, torch.sum(flow_mask))

    # Grid for upsampled real
    a = float(real_br[0]-real_tl[0])/480*1.0000001
    b = float(real_br[1]-real_tl[1])/640*1.0000001
    grid_real_h = torch.arange(int(real_tl[0]) ,int(real_br[0]) , a, device=u_map.device)[:,None].repeat(1,640)
    grid_real_w = torch.arange(int(real_tl[1]) ,int(real_br[1]) , b, device=u_map.device)[None,:].repeat(480,1)


    # Grid for upsampled ren
    a = float(ren_br[0]-ren_tl[0])/480*1.0000001
    b = float(ren_br[1]-ren_tl[1])/640*1.0000001
    c = 0
    
    grid_ren_h = torch.arange(int(ren_tl[0]) ,int(ren_br[0]) , a, device=u_map.device)[:,None].repeat(1,640)
    grid_ren_w = torch.arange(int(ren_tl[1]) ,int(ren_br[1]) , b, device=u_map.device)[None,:].repeat(480,1)
    # Calculate valid depth map for rendered image
    render_d_ind_h = torch.arange(0 ,480 , 1, device=u_map.device)[:,None].repeat(1,640)
    render_d_ind_w= torch.arange(0 ,640 , 1, device=u_map.device)[None,:].repeat(480,1)

    render_d_ind_h = torch.clamp(torch.round((render_d_ind_h - u_map).type(torch.float32)) ,0,479).type( torch.long )[flow_mask]
    render_d_ind_w = torch.clamp(torch.round((render_d_ind_w - v_map).type(torch.float32)),0,639).type( torch.long )[flow_mask] 
    index = render_d_ind_h*640 + render_d_ind_w # hacky indexing along two dimensions
    ren_d_masked  = render_d.flatten()[index]
    
    # Project depth map to the pointcloud real
    cam_scale = 10000

    anal_tensor( flow_mask, 'flow_mask')
    anal_tensor( grid_real_w, 'grid_real_w')
    anal_tensor( grid_real_h, 'grid_real_h')
    anal_tensor( u_map, 'u_map')
    
    real_pixels = torch.stack( [grid_real_w[flow_mask], grid_real_h[flow_mask], torch.ones(grid_real_h.shape, device = u_map.device,  dtype= u_map.dtype)[flow_mask]], dim=1 ).type(u_map.dtype)
    K_inv = torch.inverse(K_real.type(torch.float32)).type(u_map.dtype)
    P_real = K_inv @ real_pixels.T
    P_real = P_real * real_d[flow_mask] / cam_scale
    P_real = P_real.T
    
    
    # Project depth map to the pointcloud render
    K_ren_inv = torch.inverse(K_ren.type(torch.float32)).type(u_map.dtype)
    
    ren_pixels = torch.stack( [ grid_ren_w, 
                            grid_ren_h,
                            torch.ones(grid_ren_h.shape)], 
                        dim=2 )
    ren_pixels = ren_pixels.view(-1,3)
    P_ren_new = K_ren_inv.type(u_map.dtype) @ ren_pixels.T.type(u_map.dtype)
    P_ren_new = (P_ren_new * render_d.view(1,-1).type(u_map.dtype) / cam_scale).T
    P_ren_new = P_ren_new [index]

#     ren_pixels = torch.stack( [grid_ren_w[flow_mask] - v_map[flow_mask], 
#                             grid_ren_h[flow_mask] - u_map[flow_mask],
#                             torch.ones(grid_ren_h.shape, device = u_map.device,  dtype= u_map.dtype )[flow_mask]], 
#                             dim=1 ).type(u_map.dtype)
#     P_ren = K_ren_inv @ ren_pixels.T
#     P_ren = P_ren * ren_d_masked / cam_scale
#     P_ren = P_ren.T

    P_ren = P_ren_new
    
    # Filter the pointclouds given the depthmap
    m_ren_depth = filter_pcd_given_depthmap(P_ren, ren_d_masked )
    m_real_depth = filter_pcd_given_depthmap(P_real, real_d[flow_mask])
    m_total =  m_ren_depth * m_real_depth
    min_points = 20
    if torch.sum(m_total) < min_points:
        print('Violated filter pcd_given_depthmap min points constrain in flow_to_trafo')
        return P_real, P_ren, P_real, torch.eye(4, dtype= u_map.dtype, device=u_map.device)
    # P_ren = P_ren[m_total] 
    # P_real = P_real[m_total]
    # print('Number of points before after depth filter', P_ren.shape)
  
    # Do not transfrom to center coordinate system
    P_real_in_center = P_real                      
    P_ren_in_center = P_ren 
    #print("real")
    m_real = filter_pcd( P_real_in_center )
    #print("render")
    m_ren = filter_pcd( P_ren_in_center )
    #print("M Real left overs:", torch.sum( m_real[:,0]), m_real.shape )
    #print("M Ren left overs:", torch.sum( m_ren[:,0]), m_ren.shape )
    
    m_tot = m_real * m_ren
    if torch.sum(m_tot) < min_points:
        print('Violated filter pcd min points constrain in flow_to_trafo')
        return P_real, P_ren, P_real, torch.eye(4, dtype= u_map.dtype, device=u_map.device)

    P_real_in_center = P_real_in_center[m_tot[:,0]]
    P_ren_in_center = P_ren_in_center[m_tot[:,0]]
    #print("M Real left overs applied shape:", P_real_in_center.shape )
    #print("M Ren left overs  applied shape:", P_ren_in_center.shape )
    
    # anal_tensor(  P_real_in_center, 'P_real_in_center m_tot masked')

    # Max mean deviation
    m_new = filter_pcd_cor(P_real_in_center, P_ren_in_center)
    
    if torch.sum(m_new) < min_points:
        print('Violated filter pcd_cor min points constrain in flow_to_trafo')
        return P_real, P_ren, P_real, torch.eye(4, dtype= u_map.dtype, device=u_map.device)

    P_real_in_center = P_real_in_center[m_new] - h_real[:3,3]
    P_ren_in_center = P_ren_in_center[m_new] - h_render[:3,3]
    
    # Get transformation#
    #print('Number of points final', P_real_in_center.shape)
    pts_trafo = min( P_real_in_center.shape[0], 1000 )
    idx = torch.randperm( P_real_in_center.shape[0] )[0:pts_trafo]
    if idx.shape[0] < 10:
        print('EEEEEEEEEEEEEEEEEEEERRRRRRRRRRRRRor')
        return P_real, P_ren, P_real, torch.eye(4, dtype= u_map.dtype, device=u_map.device)

    P_real_in_center = P_real_in_center[idx]
    P_ren_in_center = P_ren_in_center[idx]
    
    
    T_res_ken = solve_transform( P_real_in_center[None].type(torch.float64 ) , P_ren_in_center.type(torch.float64 ) ).type(u_map.dtype )
    P_ren_ken = get_H( P_ren_in_center )
    P_ren_ken = (P_ren_ken @ T_res_ken[0].T)[:,:3]
    
    
    P_ren_jonas = get_H( P_ren_in_center )
    T_res_tot = torch.eye(4)
    P_ren_total = get_H( P_ren_in_center )
    
    for i in range (0,10):
        T_res_jonas = solve_transform2( P_ren_jonas[:,:3], P_real_in_center).type(u_map.dtype)[None,:,:]
        P_ren_jonas = (P_ren_jonas @ T_res_jonas[0].T)
        
        T_res_tot = T_res_tot @ T_res_jonas[0]
        
        #print(f'New {i}', torch.sum( torch.norm( P_real_in_center-P_ren_jonas[:,:3], dim=1 )))

    
    P_ren_total = (P_ren_total @ T_res_tot.T)[:,:3]
    
    #print(f'KEN ', torch.sum( torch.norm( P_real_in_center-P_ren_ken, dim=1 )))
    #print(f'Total ', torch.sum( torch.norm( P_real_in_center- P_ren_total, dim=1 )))
    
    T_res = T_res_ken
    
    # Transform the real points according to calculated transformation
    P_hr = torch.ones( (P_real_in_center.shape[0],4 ) , device=u_map.device, dtype= u_map.dtype)
    P_hr[:,:3] = P_real_in_center
    P_real_trafo = (torch.inverse( T_res[0].type(torch.float32) ).type(u_map.dtype ) @ copy.deepcopy(P_hr).T).T [:,:3]

    mask_close_to_camera = P_real_in_center[:,2] > 0.01
    #print("SUM", torch.sum(mask_close_to_camera) )
    #print("shape", mask_close_to_camera.shape)
    

    P_real_in_center = P_real_in_center[mask_close_to_camera]
    P_ren_in_center = P_ren_in_center[mask_close_to_camera]
    P_real_trafo = P_real_trafo[mask_close_to_camera]
    #print("shape P_real_trafo", P_real_trafo.shape)
    
    sub = max(1,int( P_real_in_center.shape[0]/50 ) )
    # plot_two_pcd_line(P_real_in_center[::sub].numpy(), P_ren_in_center[::sub].numpy() )
    # plot_two_pcd_line(P_real_trafo[::sub].numpy(), P_ren_in_center[::sub].numpy() )
    
    
    return P_real_in_center, P_ren_in_center, P_real_trafo, T_res[0]
#print('Okay')

tensor([[1.0000, 0.0000, 0.0000, 0.7223],
        [0.0000, 1.0000, 0.0000, 0.7223],
        [0.0000, 0.0000, 1.0000, 0.7223],
        [0.0000, 0.0000, 0.0000, 1.0000]])
tensor([[1.7223, 1.7223, 1.7223],
        [1.7223, 1.7223, 1.7223],
        [1.7223, 1.7223, 1.7223],
        ...,
        [1.7223, 1.7223, 1.7223],
        [1.7223, 1.7223, 1.7223],
        [1.7223, 1.7223, 1.7223]])


In [7]:
P_real_in_center, P_ren_in_center, P_real_trafo, gt_T_res

T_res = solve_transform( P_real_trafo[None].type(torch.float64 ) , P_ren_in_center.type(torch.float64 ) ).type(torch.float32)

# P_hr = torch.ones( (P_real_in_center.shape[0],4 ) , device="cpu", dtype= torch.float32)
# P_hr[:,:3] = P_real_in_center
# P_real_trafo2 = (torch.inverse( T_res[0].type(torch.float32) ).type(torch.float32 ) @ copy.deepcopy(P_hr).T).T [:,:3]
    
    
# plot_two_pcd_line(P_real_trafo[::sub].numpy(), P_ren_in_center[::sub].numpy() )
# plot_two_pcd_line(P_real_trafo2[::sub].numpy(), P_ren_in_center[::sub].numpy() )


NameError: name 'P_real_in_center' is not defined

In [11]:
idxmax = -1

In [37]:
# Robustness analysis: 
# Load Sample
# Predict Pose
# Calculate ADD-S
# Store array containing UniqueDesig + ADD-S Value

# Try out what happens if flow dilatation is used. 
# Maybe the normal filtering aswell. 
# Check if there is some higher error for other objects
from visu import plot_two_pcd
device = 'cpu'

desig_ls= []
adds_gt_ls = []
adds_init_ls = []
adds_erode_ls = []
adds_noise1_ls = []
adds_noise2_ls = []
visualizer = Visualizer('/home/jonfrey/Debug', None)
print('START')


K_ren = torch.tensor( dataset_train._backend.get_camera('data_syn/0019', K=True), device=device ) 
max_iter = 5
import itertools

if idxmax != -1:
    print(idxmax, "is not equal to -1 load batch")
    batch = next(itertools.islice(dataloader_train, idxmax, None))[0]
    # raise Exception

for j,ba in enumerate(dataloader_train):
    #if j != 7:
    #    continue

    if j > max_iter-1:
        break
        
    if j % 10 == 0:
        print(f"Processed {j}/{max_iter}")
#     if j != idxmax:
#         continue 
    if idxmax == -1:  
        batch = ba[0] #bann 10450   
    if idxmax != -1 and j > 1:
        break
        
    model_points = batch[4]
    idx = batch[5]  # Be carefull here the first objects starts with 0. Normally 0 is the NO object class in all other datastructures
    label = batch[7]
    real_img_original = batch[8]
    cam = batch[9]
    gt_rot_wxyz, gt_trans, unique_desig = batch[10:13] # unique_desig[1] contains the idx starting at 1 for the first object 

    log_scalars = {}
    bs = model_points.shape[0]

    # check if skip
    if batch[13] is False:
        print('Continue')
        continue

    real_img, render_img, real_d, render_d, gt_label_cropped = batch[13:18]
    pred_rot_wxyz, pred_trans, pred_points, h_render, h_real, render_img_original = batch[18:24]
    u_map, v_map, flow_mask, bb = batch[24:]
    data = torch.cat([real_img, render_img], dim=1)

    ero_in = (gt_label_cropped ==  unique_desig[1])[:,None,:,:].type(torch.float32) # BS,C,H,W
    def get_scale_for_erosion(ero_in):
        res = torch.sum ( ero_in, dim = (2,3))
        res[res < 5000] = 5
        res[res < 10000] = 10
        res[res < 30000] = 20
        res[res < 40000] = 25
        res[res < 50000] = 30
        res[res >= 50000] = 40
        return res

    t_size = get_scale_for_erosion(ero_in)
    ero_out = eroision_batch(ero_in,t_size)

    uv_gt = torch.stack( [u_map, v_map], dim=3 ).permute(0,3,1,2)
    real_tl, real_br, ren_tl, ren_br = bb 

    b = 0
    K_real = torch.tensor( [[cam[b,2],0,cam[b,0]],[b,cam[b,3],cam[b,1]],[0,0,1]], device=device )

    h_real_est = torch.eye(4,device=device)
    h_real_est[:3,:3] = quat_to_rot(pred_rot_wxyz[b][None,:], conv='wxyz', device=device)
    h_real_est[:3,3] = torch.tensor( pred_trans[b].clone().detach() ,device=device )

    ### GT Estimate ###
    typ = u_map.dtype
    indx = gt_label_cropped[0] == unique_desig[1] 
    fmt = flow_mask.dtype
    #print(f"--- Bounding Box Rendered: {ren_tl, ren_br}")
    #print ("-----------gt-------------")
    #print(f"FLOW MASK ERODE, {flow_mask.shape}")
    P_real_in_center, P_ren_in_center, P_real_trafo, gt_T_res = flow_to_trafo(
        real_br = real_br[b],
        real_tl = copy.deepcopy(real_tl[b]), 
        ren_br = copy.deepcopy(ren_br[b]), 
        ren_tl = copy.deepcopy(ren_tl[b]),
        flow_mask = copy.deepcopy(flow_mask[b]), 
        u_map = copy.deepcopy(u_map[b].type( typ )),
        v_map = copy.deepcopy(v_map[b].type( typ )), 
        K_real = copy.deepcopy(K_real.type( typ )),
        K_ren = copy.deepcopy(K_ren.type( typ )),
        real_d = copy.deepcopy(real_d[b].type( typ )),
        render_d = copy.deepcopy(render_d[b].type( typ )),
        h_real = copy.deepcopy(h_real_est.type( typ )), 
        h_render = copy.deepcopy(h_render[b].type( typ )))
    gt_h_est =  gt_T_res @ h_render[0].type(typ)

    ### Noise1 ###
    noise1 = 5
    noi = np.random.uniform(-noise1,noise1,u_map[b].shape)
    noi = torch.tensor( noi )
    noi_ = np.random.uniform(-noise1,noise1,u_map[b].shape)
    noi_ = torch.tensor( noi_ )
    #print ("-----------noise1-------------")
    _,_,_, noise1_T_res = flow_to_trafo(
        real_br = copy.deepcopy(real_br[b]),
        real_tl = copy.deepcopy(real_tl[b]), 
        ren_br = copy.deepcopy(ren_br[b]), 
        ren_tl = copy.deepcopy(ren_tl[b]),
        flow_mask = copy.deepcopy(flow_mask[b]), 
        u_map = copy.deepcopy((u_map[b]+noi).type( typ )), 
        v_map = copy.deepcopy((v_map[b]+noi_).type( typ )), 
        K_real = copy.deepcopy(K_real.type( typ )),
        K_ren = copy.deepcopy(K_ren.type( typ )),
        real_d = copy.deepcopy(real_d[b].type( typ )),
        render_d = copy.deepcopy(render_d[b].type( typ )),
        h_real = copy.deepcopy(h_real_est.type( typ )), 
        h_render = copy.deepcopy(h_render[b].type( typ )))

    noise1_h_est =  noise1_T_res @ h_render[0].type(typ)

    ### Noise2 ###
    noise2 = 10
    val1 = np.random.uniform(-noise2,noise2,(1))[0]
    noi = np.zeros( u_map[b].shape )
    noi[:,:] = val1
    noi = torch.tensor( noi )

    val1 = np.random.uniform(-noise2,noise2,(1))[0]
    noi_ = np.zeros( u_map[b].shape )
    noi_[:,:] = val1
    noi_ = torch.tensor( noi_ )
    #print (f"-----------noise2--------{u_map[b].shape}-----")
    _,_,_, noise2_T_res = flow_to_trafo(
        real_br = copy.deepcopy(real_br[b]),
        real_tl = copy.deepcopy(real_tl[b]), 
        ren_br = copy.deepcopy(ren_br[b]), 
        ren_tl = copy.deepcopy(ren_tl[b]),
        flow_mask = copy.deepcopy(flow_mask[b]), 
        u_map = copy.deepcopy((u_map[b]+noi).type( typ )), 
        v_map = copy.deepcopy((v_map[b]+noi_).type( typ )), 
        K_real = copy.deepcopy(K_real.type( typ )),
        K_ren = copy.deepcopy(K_ren.type( typ )),
        real_d = copy.deepcopy(real_d[b].type( typ )),
        render_d = copy.deepcopy(render_d[b].type( typ )),
        h_real = copy.deepcopy(h_real_est.type( typ )), 
        h_render = copy.deepcopy(h_render[b].type( typ )))
    noise2_h_est =  noise2_T_res @ h_render[0].type(typ)


    ### Erode Label ###
    #print ("-----------Eroded-------------")
    flow_mask  = (flow_mask * ero_out.type(torch.float32)).type(fmt)[:,0]
    #print(f"FLOW MASK ERODE, {flow_mask.shape}")
    _,_,_, ero_T_res = flow_to_trafo(
        real_br = copy.deepcopy(real_br[b]),
        real_tl = copy.deepcopy(real_tl[b]), 
        ren_br = copy.deepcopy(ren_br[b]), 
        ren_tl = copy.deepcopy(ren_tl[b]),
        flow_mask = copy.deepcopy(flow_mask[b]), 
        u_map = copy.deepcopy((u_map[b]).type( typ )), 
        v_map = copy.deepcopy((v_map[b]).type( typ )), 
        K_real = copy.deepcopy(K_real.type( typ )),
        K_ren = copy.deepcopy(K_ren.type( typ )),
        real_d = copy.deepcopy(real_d[b].type( typ )),
        render_d = copy.deepcopy(render_d[b].type( typ )),
        h_real = copy.deepcopy(h_real_est.type( typ )), 
        h_render = copy.deepcopy(h_render[b].type( typ )))
    ero_h_est =  ero_T_res @ h_render[0].type(typ) # set rotation




    mask = (flow_mask == True)

    # Target Model-points Transformed
    p = model_points.shape[1]
    target = torch.bmm( model_points, torch.transpose(h_real[:,:3,:3], 1,2 ) ) + h_real[:,:3,3][:,None,:].repeat(1,p,1)

    # Compute ADD-S
    adds_res_gt_flow = criterion_adds(target[b][None], model_points[b][None], idx[b][None], H = gt_h_est[None].type( target.dtype) )
    adds_res_gt_flow_eroded = criterion_adds(target[b][None], model_points[b][None], idx[b][None], H = ero_h_est[None].type( target.dtype) )
    adds_res_noise1 = criterion_adds(target[b][None], model_points[b][None], idx[b][None], H = noise1_h_est[None].type( target.dtype) )
    adds_res_noise2 = criterion_adds(target[b][None], model_points[b][None], idx[b][None], H = noise2_h_est[None].type( target.dtype) )
    adds_init = criterion_adds(target[b][None], model_points[b][None], idx[b][None], H = h_real_est[None].type( target.dtype))

    #print(f'Iteration {j}, GT', float(adds_res_gt_flow), 'ERRO', float( adds_res_gt_flow_eroded))
    #print(f'Iteration {j}, No1' ,float(adds_res_noise1), 'No2', float(adds_res_noise2), 'Init', float(adds_init))
    desig_ls.append(unique_desig)
    adds_gt_ls.append(float(adds_res_gt_flow.detach()))
    adds_erode_ls.append(float(adds_res_gt_flow_eroded.detach()))
    adds_init_ls.append(float(adds_init.detach()))
    adds_noise1_ls.append(float(adds_res_noise1.detach()))
    adds_noise2_ls.append(float(adds_res_noise2.detach()))

    plot1 = False
    if adds_erode_ls[-1] > 0.02 :
        print(f'Iteration {j}, GT', adds_gt_ls[-1], 'ERRODED', float( adds_res_gt_flow_eroded)   )
        #plot_two_pcd_line(P_real_trafo[::sub], P_ren_in_center[::sub] )
        no = torch.norm(P_real_trafo- P_ren_in_center, dim=1)
        print( P_real_trafo.shape )
        print( f'number of point with a distance greater 0.2 {torch.sum( no>0.2 )} ' )
        print( f'number of point with a distance greater 0.02 {torch.sum( no>0.02 )} ' )
        print( f'number of point with a distance smaller 0.02 {torch.sum( no<0.02 )} ' )
        
        
        plot1 = False
        
        
    if idxmax != -1:
        plot1 = True
        
    if plot1:
        print("Real depth map cropped")
        real_depth_img = Drawer().disp_img_1d(real_d[b].numpy(),ret=True)
        real_depth_img = np.repeat( real_depth_img[:,:,None],3, axis=2)
        print("Render depth map cropped")
        render_depth_img = Drawer().disp_img_1d(render_d[b].numpy(),ret=True)
        render_depth_img = np.repeat( render_depth_img[:,:,None],3, axis=2)
        Drawer().disp_img_1d(flow_mask[0],ret=True)


        sub = max(1,int( P_real_in_center.shape[0]/100 ) )
        plot_two_pcd_line(P_real_in_center[::sub].numpy(), P_ren_in_center[::sub].numpy() )
        plot_two_pcd_line(P_real_trafo[::sub].numpy(), P_ren_in_center[::sub].numpy() )

        print(f"Real Image, Estimated Points given GT Flow {P_real_in_center.shape}")
        visualizer.plot_estimated_pose( tag = f"_",
                        epoch = 1,
                        img= real_img_original[b].cpu().numpy(),
                        points = copy.deepcopy( P_real_in_center.cpu().numpy()),
                        store = False,
                        jupyter=True,
                        K = K_real.cpu().numpy(),
                        H = np.eye(4),
                        method='def')
        print("Real Image Cropped, Estimated Points given GT Flow")
        visualizer.plot_estimated_pose_on_bb( tag = f"_",
                        epoch = 1,
                        img= real_depth_img,
                        tl = real_tl[0],
                        br = real_br[0],
                        points = copy.deepcopy( P_real_in_center.cpu().numpy()),
                        store = False,
                        jupyter=True,
                        K = K_real.cpu().numpy(),
                        H = np.eye(4),
                        method='def')

        fil = label == unique_desig[1]    
        real_img_original[b][ fil[0][:,:,None].repeat(1,1,3) ] = 255
        print("Real Image, Estimated Points given GT Flow with label is white")
        visualizer.plot_estimated_pose( tag = f"_",
                        epoch = 1,
                        img= real_img_original[b].cpu().numpy(),
                        points = copy.deepcopy( P_real_in_center.cpu().numpy()),
                        store = False,
                        jupyter=True,
                        K = K_real.cpu().numpy(),
                        H = np.eye(4),
                        method='def')

        print("Render Image, Rendered Points not transformed reprojected")
        visualizer.plot_estimated_pose( tag = f"_",
                            epoch = 1,
                            img= render_img_original[b].cpu().numpy(),
                            points = copy.deepcopy(P_ren_in_center.cpu().numpy()),
                            store = False,
                            jupyter=True,
                            K = K_ren.cpu().numpy(),
                            H = np.eye(4),
                            method='def')
        print("Render Image Cropped, Rendered Points not transformed reprojected")
        visualizer.plot_estimated_pose_on_bb( tag = f"_",
                            epoch = 1,
                            img= render_depth_img,
                            points = copy.deepcopy(P_ren_in_center.cpu().numpy()),
                            tl = ren_tl[0],
                            br = ren_br[0],
                            store = False,
                            jupyter=True,
                            K = K_ren.cpu().numpy(),
                            H = np.eye(4),
                            method='def')
        print("Corrospondence")

        print(real_img.shape, render_img.shape,flow_mask.shape, u_map.shape)
        visualizer.plot_corrospondence(tag=f'_',
                                           epoch=0,
                                            u_map=u_map[0], 
                                            v_map=v_map[0], 
                                            flow_mask=flow_mask[0], 
                                            real_img=real_img[0], 
                                            render_img=render_img[0],
                                            store=False,
                                            jupyter=True,
                                            coloful=True,
                                            method='def',
                                            res_h=1,
                                            res_w=1)

        
print(f'Result, GT Avg', sum(adds_gt_ls)/len(adds_gt_ls), 'Errode Avg', sum(adds_erode_ls)/len(adds_erode_ls) )


START
Processed 0/5
Iteration 0, GT 0.37210410833358765 ERRODED 0.26329484581947327
torch.Size([0, 3])
number of point with a distance greater 0.2 0 
number of point with a distance greater 0.02 0 
number of point with a distance smaller 0.02 0 


/home/jonfrey/miniconda3/envs/track_latest/lib/python3.7/site-packages/ipykernel_launcher.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Iteration 1, GT 0.33457842469215393 ERRODED 0.30314794182777405
torch.Size([2, 3])
number of point with a distance greater 0.2 0 
number of point with a distance greater 0.02 2 
number of point with a distance smaller 0.02 0 
Iteration 2, GT 0.2000366598367691 ERRODED 0.19722352921962738
torch.Size([3, 3])
number of point with a distance greater 0.2 0 
number of point with a distance greater 0.02 3 
number of point with a distance smaller 0.02 0 
Iteration 3, GT 0.2473229169845581 ERRODED 0.21554017066955566
torch.Size([24, 3])
number of point with a distance greater 0.2 0 
number of point with a distance greater 0.02 4 
number of point with a distance smaller 0.02 20 
Iteration 4, GT 0.29877397418022156 ERRODED 0.31761690974235535
torch.Size([438, 3])
number of point with a distance greater 0.2 0 
number of point with a distance greater 0.02 438 
number of point with a distance smaller 0.02 0 
Result, GT Avg 0.29056321680545805 Errode Avg 0.25936467945575714


In [ ]:
import pandas
data = np.array( [adds_gt_ls,adds_init_ls,adds_erode_ls,adds_noise1_ls,adds_noise2_ls] ).T
df = pandas.DataFrame( data, columns = ['ADD-S GT','ADD-S INITAL','ADD-S Eroded Mask','ADD-S Noise All Random','ADD-S Noise Same Random'] )
print(df)
print("MEAN", df.mean() )
print("STD", df.std() )
print("MAX", df.max(), df.idxmax() )
idxmax = df.idxmax()[0]
print(idxmax)

In [ ]:
idxmax = -1
#batch[0][0]

In [ ]:
cropped_comp = np.concatenate( [real_img.cpu().numpy(), render_img.cpu().numpy() ], axis=1).astype(np.uint8)
cropped_comp_img = Image.fromarray(cropped_comp)

In [ ]:

Nc = 256
cmap = plt.cm.get_cmap('gist_rainbow', Nc)
cmaplist = [cmap(i) for i in range(cmap.N)]

def plot(mask):
    min_val = torch.min( mask )
    max_val = float( max(1,torch.max( mask )) )
    mask = torch.clamp( (mask-min_val) / (max_val-min_val)*255 ,0,255)
    
    data_depth = np.zeros((480,640,4), dtype=np.uint8)
    data_depth[:,:,3] = 255
#     print(val.shape)
    for i in range(480):
        for j in range(640):
#             print(int( val[i,j]),  cmaplist[ int(val[i,j])])
            data_depth[i,j,:4] = np.array( cmaplist[ int(mask[i,j])] )*255
    data_depth[:,:,3] = 255
    data_depth[:,:,3][label==2] = 255
    img_depth = Image.fromarray(data_depth, 'RGBA')
    display(img_depth)
    
def plot_depth(depth):
    min_val = torch.min( depth[depth!=0] )
    max_val = float( max(1,torch.max( depth )) )
    mask = torch.clamp( (depth-min_val) / (max_val-min_val)*255 ,0,255)
    
    data_depth = np.zeros((480,640,4), dtype=np.uint8)
    data_depth[:,:,3] = 255
#     print(val.shape)
    for i in range(480):
        for j in range(640):
#             print(int( val[i,j]),  cmaplist[ int(val[i,j])])
            data_depth[i,j,:4] = np.array( cmaplist[ int(mask[i,j])] )*255
    data_depth[:,:,3] = 255
    data_depth[:,:,3][label==2] = 255
    
    img_depth = Image.fromarray(data_depth, 'RGBA')
    
    display(img_depth)

In [ ]:
from helper import BoundingBox

bb = BoundingBox(p1 = torch.tensor([100,100], dtype=torch.float32),p2 = torch.tensor([490,670], dtype=torch.float32))
img = torch.tensor(render_depth_img).type(torch.float32)
#crop_ren = bb.crop(img).unsqueeze(0)
#crop_ren.shape

def crop(img, tl,br, max_height=480, max_width=640, scale=False, mode='nearest'):

    h = img.shape[0]
    w = img.shape[1]

    img_pad = torch.zeros((int(h + 2*max_height), int(w + 2*max_width), img.shape[2]))
    
    img_pad[max_height:max_height+h,max_width:max_width+w] = img
    res = img_pad[ int(max_height+tl[0]) : int(max_height+br[0]), int(max_width+tl[1]) : int(max_width+br[1])] # H W C
    if scale:
        res = res.permute(2,0,1)[None] #BS C H W
        res  = torch.nn.functional.interpolate(res, size=(480,640), mode=mode)  
        res = res[0].permute(1,2,0) # H W C
        
    return res
 

res = crop(img, bb.tl, bb.br, scale = True, mode='nearest')

def plot_torch(t):
    if t.shape[0] == 3:
        t = t.permute(1,2,0)
    t = t/float(torch.max(t)/255)
    
    t = np.uint8(t.numpy())
    display( Image.fromarray(t ) )
print(res.shape)
plot_torch(res)

In [ ]:
def solve_transform(keypoints, gt_keypoints):
    """
    keypoints: N x K x 3
    gt_keypoints: K x 3
    return: N x 4 x 4 transformation matrix
    """
    try:
        keypoints = keypoints.clone()
        gt_keypoints = gt_keypoints.clone()
        N, K, _ = keypoints.shape
        center = keypoints.mean(dim=1)
        gt_center = gt_keypoints.mean(dim=0)
        keypoints -= center[:, None, :]
        gt_keypoints -= gt_center[None]
        matrix = keypoints.transpose(2, 1) @ gt_keypoints[None]
        U, S, V = torch.svd(matrix)
        Vt = V.transpose(2, 1)
        Ut = U.transpose(2, 1)

        d = (V @ Ut).det()
        I = torch.eye(3, 3, dtype=gt_center.dtype, device= keypoints.device)[None].repeat(N, 1, 1)
        I[:, 2, 2] = d.clone()

        R = U @ I @ Vt
        T = torch.zeros(N, 4, 4, dtype=gt_center.dtype, device= keypoints.device)
        T[:, 0:3, 0:3] = R
        T[:, 0:3, 3] = center[None] - (R @ gt_center[None :, None])[:, :, 0]
        T[:, 3, 3] = 1.0

        return T
    except RuntimeError as error:
        import ipdb; ipdb.set_trace()
        print("Something went wrong")

        # costume implementation 
def solve_transform2(A,B):
    if A.shape[0] > B.shape[0]:
        x=torch.arange(A.shape[0],device=A.device)
        out = torch.randperm(x.numel(),device=A.device)[:B.shape[0]]
        A = torch.index_select(A, 0, out)
    if A.shape[0] < B.shape[0]:
        x=torch.arange(B.shape[0],device=A.device)
        out = torch.randperm(x.numel(),device=A.device)[:A.shape[0]]
        B = torch.index_select(B, 0, out)

    #A = torch.choice(A,B.shape[0])

    assert A.shape == B.shape

    m = A.shape[1]
    centroid_A = torch.mean(A, dim=0)
    centroid_B = torch.mean(B, dim=0)

    AA = (A - centroid_A)
    BB = (B - centroid_B)
    H = AA.transpose(0,1) @ BB
    U, S, Vt = torch.svd(H)
    R = Vt @ U.transpose(0,1)
    if torch.det(R) < 0:
        Vt[m-1,:] *= -1
        R = Vt.transpose(0,1) @ U.transpose(0,1)

    # translation
    t = centroid_B - (R @ centroid_A)
    # homogeneous transformation
    T = torch.eye(m+1, device= A.device)
    T[:m, :m] = R
    T[:m, m] = t
    return T

NR = 1000
DIM = 3
A = torch.rand( (NR,DIM), dtype= torch.float32)
B = torch.rand( (NR,DIM), dtype= torch.float32)
T = solve_transform2(A,B)
T_ken = solve_transform(B[None,:,:],A)[0]
print('Jonas\n', T, '\n \nKens Impl \n',T_ken)
def get_H(pcd):
    pcd_ret = torch.ones( (pcd.shape[0],pcd.shape[1]+1),device=pcd.device, dtype=pcd.dtype )
    pcd_ret[:,:3] = pcd
    return pcd_ret

# A_hom = get_H(A)
# A_hom2 = A_hom @ T.T
# print(A_hom2[:,:3])